# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Cleaning database to get only found cities
vacations_df = pd.read_csv("output_data/temperature_by_city.csv")
vacations_df = vacations_df.drop(columns="Unnamed: 0")
vacations_df = vacations_df.loc[vacations_df["Temperature"]!="Not Found"]
vacations_df["Temperature"] = vacations_df["Temperature"].astype(float)
vacations_df["Humidity"] = vacations_df["Humidity"].astype(float)
vacations_df["Wind Speed"] = vacations_df["Wind Speed"].astype(float)
vacations_df.head()

,Country,City,Temperature,Max Temperature,Wind Speed,Cloudiness,Humidity,Latitude,Longitude
0,PT,vila franca do campo,18.00,18,2.60,75,59.0,37.72,-25.43
1,CA,norman wells,-7.00,-7,2.10,40,79.0,65.28,-126.83
2,LY,sabha,21.55,21.55,4.01,0,23.0,27.04,14.43
3,CF,birao,23.87,23.87,1.67,32,49.0,10.28,22.79
4,PF,rikitea,24.20,24.2,7.49,100,83.0,-23.12,-134.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = vacations_df[["Latitude","Longitude"]].astype(float)
humidity = vacations_df["Humidity"].astype(float)
max_int = vacations_df["Humidity"].astype(float).max()

In [4]:
fig = gmaps.figure(center=(50,10),zoom_level=5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_int,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
temp_df = vacations_df.loc[(vacations_df["Temperature"] < 20) & (vacations_df["Temperature"] > 10) & (vacations_df["Humidity"] < 50) & (vacations_df["Wind Speed"] < 3)]
temp_df

,Country,City,Temperature,Max Temperature,Wind Speed,Cloudiness,Humidity,Latitude,Longitude
178,US,saint george,16.00,16,1.92,1,9.0,37.1,-113.58
194,IR,eqbaliyeh,14.52,17,0.85,0,44.0,35.3,51.54
252,IN,sanaur,16.50,16.5,1.68,0,31.0,30.3,76.46
287,IN,nathdwara,18.50,18.5,1.20,0,27.0,24.93,73.82
440,IR,mehran,11.00,11,2.10,0,25.0,33.12,46.16
500,SA,najran,16.00,16,2.10,0,25.0,17.49,44.13
510,IR,ardakan,13.00,13,2.10,0,24.0,32.31,54.02
521,US,weiser,13.61,16,0.68,1,25.0,44.25,-116.97
564,IR,dehloran,19.69,19.69,1.18,0,26.0,32.69,47.27


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = temp_df
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df.head()

<ipython-input-6-0531039c136a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
<ipython-input-6-0531039c136a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Address"] = ""


,Country,City,Temperature,Max Temperature,Wind Speed,Cloudiness,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
178,US,saint george,16.00,16,1.92,1,9.0,37.1,-113.58,,
194,IR,eqbaliyeh,14.52,17,0.85,0,44.0,35.3,51.54,,
252,IN,sanaur,16.50,16.5,1.68,0,31.0,30.3,76.46,,
287,IN,nathdwara,18.50,18.5,1.20,0,27.0,24.93,73.82,,
440,IR,mehran,11.00,11,2.10,0,25.0,33.12,46.16,,


In [7]:
# set parameters
params = {
    "type":"hotel",
    "keyword":"hotel",
    "radius":5000,
    "key":g_key
}

url_search = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [8]:
hotel_df

,Country,City,Temperature,Max Temperature,Wind Speed,Cloudiness,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
178,US,saint george,16.00,16,1.92,1,9.0,37.1,-113.58,,
194,IR,eqbaliyeh,14.52,17,0.85,0,44.0,35.3,51.54,,
252,IN,sanaur,16.50,16.5,1.68,0,31.0,30.3,76.46,,
287,IN,nathdwara,18.50,18.5,1.20,0,27.0,24.93,73.82,,
440,IR,mehran,11.00,11,2.10,0,25.0,33.12,46.16,,
500,SA,najran,16.00,16,2.10,0,25.0,17.49,44.13,,
510,IR,ardakan,13.00,13,2.10,0,24.0,32.31,54.02,,
521,US,weiser,13.61,16,0.68,1,25.0,44.25,-116.97,,
564,IR,dehloran,19.69,19.69,1.18,0,26.0,32.69,47.27,,


In [9]:
print(f"""Showing the hotel information near to the cities provided...
    ----------------------------------------------------------------------
    """)

# Starting loop with iterrows to get the value of each lat and lng
for index, location in hotel_df.iterrows():
    lat = location["Latitude"]
    lon = location["Longitude"]
    
    # Doing the request 
    params["location"] = f"{lat},{lon}"
    response = requests.get(url_search, params=params).json()
    # Trying to get the first hotel name and address
    try:
        hotel_name = response["results"][0]["name"]
        vicinity= response["results"][0]["vicinity"]
        print(f"""The first result shown for city {location['City']} is {hotel_name}
        The location is: {vicinity}""")
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        hotel_df.loc[index, "Hotel Address"] = vicinity
    except:
        print(f"""There were no hotels found on city {location["City"]}
        Continuing with next search...""")
        hotel_df.loc[index, "Hotel Address"] = "Not Found"
        hotel_df.loc[index, "Hotel Name"] = "Not Found"

print("The search is complete, the results have been stored!")
        

Showing the hotel information near to the cities provided...
    ----------------------------------------------------------------------
    
The first result shown for city saint george is The Advenire, Autograph Collection
        The location is: 25 W St George Blvd, St. George


C:\Users\Carlo\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


There were no hotels found on city eqbaliyeh
        Continuing with next search...
The first result shown for city sanaur is Hotel Eqbal Inn
        The location is: Rajpura Rd, SST Nagar, Patiala
The first result shown for city nathdwara is श्री Vallabh Vilas Lords Plaza
        The location is: Adjacent to Shrinathji Temple,Moti Mahal entrance, NH 162 Ext, Nathdwara
The first result shown for city mehran is مجتمع رفاهی نور سپاه
        The location is: Mehran
There were no hotels found on city najran
        Continuing with next search...
The first result shown for city ardakan is Kheshtomah Ecolodge
        The location is: Ardakan
The first result shown for city weiser is State Street Motel
        The location is: 1279 State St, Weiser
The first result shown for city dehloran is هتل سعید
        The location is: Dehloran
The search is complete, the results have been stored!


In [10]:
hotel_df

,Country,City,Temperature,Max Temperature,Wind Speed,Cloudiness,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
178,US,saint george,16.00,16,1.92,1,9.0,37.1,-113.58,"The Advenire, Autograph Collection","25 W St George Blvd, St. George"
194,IR,eqbaliyeh,14.52,17,0.85,0,44.0,35.3,51.54,Not Found,Not Found
252,IN,sanaur,16.50,16.5,1.68,0,31.0,30.3,76.46,Hotel Eqbal Inn,"Rajpura Rd, SST Nagar, Patiala"
287,IN,nathdwara,18.50,18.5,1.20,0,27.0,24.93,73.82,श्री Vallabh Vilas Lords Plaza,"Adjacent to Shrinathji Temple,Moti Mahal entra..."
440,IR,mehran,11.00,11,2.10,0,25.0,33.12,46.16,مجتمع رفاهی نور سپاه,Mehran
500,SA,najran,16.00,16,2.10,0,25.0,17.49,44.13,Not Found,Not Found
510,IR,ardakan,13.00,13,2.10,0,24.0,32.31,54.02,Kheshtomah Ecolodge,Ardakan
521,US,weiser,13.61,16,0.68,1,25.0,44.25,-116.97,State Street Motel,"1279 State St, Weiser"
564,IR,dehloran,19.69,19.69,1.18,0,26.0,32.69,47.27,هتل سعید,Dehloran


In [11]:
hotel_df["Latitude"] = hotel_df["Latitude"].astype(float)
hotel_df["Longitude"] = hotel_df["Longitude"].astype(float)

<ipython-input-11-dd7002029d42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Latitude"] = hotel_df["Latitude"].astype(float)
<ipython-input-11-dd7002029d42>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Longitude"] = hotel_df["Longitude"].astype(float)


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))